<a href="https://colab.research.google.com/github/holazzer/nb/blob/main/nb/Gemma3-1B-sqxx-sft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [2]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.5: Fast Gemma3 patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


We now add LoRA adapters so we only need to update a small amount of parameters!

In [3]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 64,           # Larger = higher accuracy, but might overfit
    lora_alpha = 64,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [4]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

custom sqxx dataset

In [5]:
import json
from tqdm import tqdm

sqxx = []

with open('sqxx-sft.jsonl', 'r', encoding='utf-8') as ff:
    for line in ff:
        d = json.loads(line)
        sqxx.append(d)

In [6]:
from unsloth.chat_templates import standardize_data_formats

In [7]:
s2 = standardize_data_formats(sqxx)

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`. We remove the `<bos>` token using removeprefix(`'<bos>'`) since we're finetuning. The Processor will add this token before training and the model expects only one.

In [8]:
def formatting_prompts_func(d):
   convo = [{"role": "user", "content": d['src']}, {"role": "assistant", "content": d['tgt']}]
   texts = tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>')
   return { "text" : texts, }

s2 = list(map(formatting_prompts_func, sqxx))

Let's see how the chat template did! Notice there is no `<bos>` token as the processor tokenizer will be adding one.

In [23]:
s2[100]

{'text': '<start_of_turn>user\n怎样评价《睡前消息》第570期？<end_of_turn>\n<start_of_turn>model\n<p>评论和弹幕表明，和自己的执念相比，很多人没有那么爱传统。</p><p>所以社会化抚养的前提，就是考察每个父母的抚养能力，如果不是因为天然条件而产生的不合理养育，就要坚决剥夺。</p><p>类似于豫章学院之类的审判，每个星期，每个县都要举行一次，用类似的标准审查父母就行了。</p><p>否则你们怎么好意思送豫章书院的经营者进监狱的？</p><figure><noscript><img/></noscript><img/></figure><p></p><end_of_turn>\n'}

In [9]:
from datasets import Dataset

ds = Dataset.from_list(s2)

In [10]:
ds

Dataset({
    features: ['text'],
    num_rows: 1211
})

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [11]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = ds,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 5, # Set this for 1 full training run.
        #max_steps = 5000,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
        dataset_num_proc=2,
    ),
)

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.


Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1211 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [12]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/1211 [00:00<?, ? examples/s]

Let's verify masking the instruction part is done! Let's print the 100th row again.  Notice how the sample only has a single `<bos>` as expected!

In [13]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><start_of_turn>user\n怎样评价《睡前消息》第570期？<end_of_turn>\n<start_of_turn>model\n<p>评论和弹幕表明，和自己的执念相比，很多人没有那么爱传统。</p><p>所以社会化抚养的前提，就是考察每个父母的抚养能力，如果不是因为天然条件而产生的不合理养育，就要坚决剥夺。</p><p>类似于豫章学院之类的审判，每个星期，每个县都要举行一次，用类似的标准审查父母就行了。</p><p>否则你们怎么好意思送豫章书院的经营者进监狱的？</p><figure><noscript><img/></noscript><img/></figure><p></p><end_of_turn>\n'

Now let's print the masked out example - you should see only the answer is present:

In [32]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                      <p>评论和弹幕表明，和自己的执念相比，很多人没有那么爱传统。</p><p>所以社会化抚养的前提，就是考察每个父母的抚养能力，如果不是因为天然条件而产生的不合理养育，就要坚决剥夺。</p><p>类似于豫章学院之类的审判，每个星期，每个县都要举行一次，用类似的标准审查父母就行了。</p><p>否则你们怎么好意思送豫章书院的经营者进监狱的？</p><figure><noscript><img/></noscript><img/></figure><p></p><end_of_turn>\n'

In [14]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
1.512 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,211 | Num Epochs = 5 | Total steps = 380
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 52,183,040/1,000,000,000 (5.22% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
1,5.057400
2,4.842400
3,4.957500
4,4.454000
5,4.187700
6,4.254400
7,3.693600
8,3.785000
9,3.744900
10,3.600500


In [16]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2877.6628 seconds used for training.
47.96 minutes used for training.
Peak reserved memory = 12.221 GB.
Peak reserved memory for training = 10.709 GB.
Peak reserved memory % of max memory = 82.905 %.
Peak reserved memory for training % of max memory = 72.648 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [52]:
messages = [{
    "role": "user",
    "content": "Continue the sequence: 1, 1, 2, 3, 5, 8,"
}]

tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
)

tensor([[     2,    105,   2364,    107,  34230,    506,   7501, 236787, 236743,
         236770, 236764, 236743, 236770, 236764, 236743, 236778, 236764, 236743,
         236800, 236764, 236743, 236810, 236764, 236743, 236828, 236764,    106,
            107,    105,   4368,    107]])

In [30]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": "福建莆田出现90.8万彩礼骗婚案例，会进一步降低结婚率吗？"
}]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    text,
    max_new_tokens = 1024, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\n福建莆田出现90.8万彩礼骗婚案例，会进一步降低结婚率吗？<end_of_turn>\n<start_of_turn>model\n<p>最起码中奖者也不会继续发钱。毕竟这些都过去十年的事情，你发个红包就是真钱，肯定没法偷。除非你把投资策略改变到发富富人也可以的，不然给每个发钱中奖的人发100%的信贷资金，再算发钱的概率。这就是个黑超大黑的游戏，和福州1990年代的犯罪率基本一样。</p><p>问题是，福州现在已经无违法信贷资金了，但是仍然有很多人想着要偷工作检。</p><figure><noscript><img/></noscript><img/></figure><p></p><end_of_turn>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Why is the sky blue?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Okay, let's break down why the sky is blue! It's a fascinating phenomenon that boils down to a combination of physics and light. Here's the explanation:

**1. Sunlight and its Colors:**

* Sunlight, which appears white to us, is actually made up of *all* the


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [20]:
model.save_pretrained("gemma-3-1b-sqxx")  # Local saving
tokenizer.save_pretrained("gemma-3-1b-sqxx")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

('gemma-3-1b-sqxx/tokenizer_config.json',
 'gemma-3-1b-sqxx/special_tokens_map.json',
 'gemma-3-1b-sqxx/chat_template.jinja',
 'gemma-3-1b-sqxx/tokenizer.model',
 'gemma-3-1b-sqxx/added_tokens.json',
 'gemma-3-1b-sqxx/tokenizer.json')

In [21]:
!ls -lh gemma-3-1b-sqxx

total 237M
-rw-r--r-- 1 root root 1.1K Jun 24 04:01 adapter_config.json
-rw-r--r-- 1 root root 200M Jun 24 04:01 adapter_model.safetensors
-rw-r--r-- 1 root root   35 Jun 24 04:01 added_tokens.json
-rw-r--r-- 1 root root 1.5K Jun 24 04:01 chat_template.jinja
-rw-r--r-- 1 root root 5.0K Jun 24 04:01 README.md
-rw-r--r-- 1 root root  670 Jun 24 04:01 special_tokens_map.json
-rw-r--r-- 1 root root 1.2M Jun 24 04:01 tokenizer_config.json
-rw-r--r-- 1 root root  32M Jun 24 04:01 tokenizer.json
-rw-r--r-- 1 root root 4.5M Jun 24 04:01 tokenizer.model


In [22]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
!ls -lh drive/MyDrive/colab-sqxx/gemma-3-1b-sqxx/

total 237M
-rw------- 1 root root 1.1K Jun 24 04:01 adapter_config.json
-rw------- 1 root root 200M Jun 24 04:01 adapter_model.safetensors
-rw------- 1 root root   35 Jun 24 04:01 added_tokens.json
-rw------- 1 root root 1.5K Jun 24 04:01 chat_template.jinja
-rw------- 1 root root 5.0K Jun 24 04:01 README.md
-rw------- 1 root root  670 Jun 24 04:01 special_tokens_map.json
-rw------- 1 root root 1.2M Jun 24 04:01 tokenizer_config.json
-rw------- 1 root root  32M Jun 24 04:01 tokenizer.json
-rw------- 1 root root 4.5M Jun 24 04:01 tokenizer.model


Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "What is Gemma-3?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Okay, let's break down what Gemma-3 is. It's a fascinating development in the world of AI, and here's a comprehensive overview:

**1. What it is:**

* **A Family of Open-Weight Language Models:** Gemma-3 isn't just *one* model


### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
